In [64]:
from datetime import datetime
import os
import pandas as pd

# Clean flights data

In [47]:
# Merge all flight data
destinations = ['boston', 'nyc', 'dc']

for dest in destinations:
    flight_csvs = [x for x in os.listdir(os.path.join('data', dest)) if 'flights' in x]
    flight_data = [pd.read_csv(os.path.join('data', dest, x)) for x in flight_csvs]
    all_flights = pd.concat(flight_data)
    all_flights.to_csv(os.path.join('data', dest, 'flights.csv'))

/Users/lucy/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [48]:
flight_data_all = [pd.read_csv(os.path.join('data', x, 'flights.csv')) for x in destinations]
flights = pd.concat(flight_data_all)

In [49]:
flights.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,destination,duration,origination,price,time
0,0,0,NaN,11-03-2019,Philadelphia,1h 36m,Boston,$94,5:00 AM – 6:36 AM
1,1,0,NaN,11-03-2019,Philadelphia,1h 36m,Boston,$94,6:00 AM – 7:36 AM
2,2,0,NaN,11-03-2019,Philadelphia,1h 39m,Boston,$94,7:38 AM – 9:17 AM
3,3,0,NaN,11-03-2019,Philadelphia,1h 41m,Boston,$94,8:03 AM – 9:44 AM
4,4,0,NaN,11-03-2019,Philadelphia,1h 37m,Boston,$104,9:38 PM – 11:15 PM


In [50]:
flights = flights[['time', 'duration', 'price', 'date', 'origination', 'destination']]

In [51]:
flights['depart'] = [x.split('–')[0] for x in flights['time']]
flights['arrive'] = [x.split('–')[-1] for x in flights['time']]

In [52]:
flights['mode'] = 'plane'
flights = flights[['duration', 'price', 'date', 'origination', 'destination', 'depart', 'arrive', 'mode']]

In [53]:
flights.to_csv(os.path.join('data', 'all_flights.csv'), index=False)

# Clean bus/train data

In [54]:
# Merge all bus/train data
destinations = ['boston', 'nyc', 'dc']

for dest in destinations:
    bus_train_csvs = [x for x in os.listdir(os.path.join('data', dest)) if 'bus_train' in x]
    bus_train_data = [pd.read_csv(os.path.join('data', dest, x)) for x in bus_train_csvs]
    all_bus_trains = pd.concat(bus_train_data)
    all_bus_trains.to_csv(os.path.join('data', dest, 'bus_train.csv'))

/Users/lucy/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [55]:
bus_train_all = [pd.read_csv(os.path.join('data', x, 'bus_train.csv')) for x in destinations]
bus_train = pd.concat(bus_train_all)

In [56]:
bus_train['mode'] = ['train' if ('Amtrak') in x else 'bus' for x in bus_train['carrier']]

In [57]:
bus_train['arrive'] = [x.split('+1')[0] for x in bus_train['arrive']]

In [58]:
bus_train = bus_train[['duration', 'price', 'date', 'origination', 'destination', 'depart', 'arrive', 'mode']]

In [59]:
bus_train.to_csv('data/all_bus_train.csv', index=False)

# Merge and clean all data

In [60]:
travel = pd.concat([flights, bus_train])

In [78]:
minute_durations = travel[~travel['duration'].str.contains('h')].copy()
minute_durations['duration'] = [int(x.rstrip('m')) for x in minute_durations['duration']]

In [85]:
hour_durations = travel[travel['duration'].str.contains('h')].copy()
dt_obj = [datetime.strptime(x, '%Hh %Mm') for x in hour_durations['duration']]
hour_durations['duration'] = [x.hour*60 + x.minute for x in dt_obj]

In [86]:
travel_clean = pd.concat([minute_durations, hour_durations])

In [87]:
travel_clean.head()

,duration,price,date,origination,destination,depart,arrive,mode
3,53,$390,11-03-2019,DC,Philadelphia,7:13 PM,8:06 PM,plane
15,56,$390,10-17-2019,Philadelphia,DC,7:59 AM,8:55 AM,plane
16,53,$390,10-17-2019,Philadelphia,DC,12:25 PM,1:18 PM,plane
17,51,$390,10-17-2019,Philadelphia,DC,4:50 PM,5:41 PM,plane
18,56,$390,10-17-2019,Philadelphia,DC,5:50 PM,6:46 PM,plane


In [ ]:
travel['arrival_hour'] = 

In [90]:
travel_clean.to_csv('data/travels.csv')